In [1]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff


In [2]:
yandex_russia = pd.read_csv('yandex_russia.csv')

yandex_russia.rename(columns = {'Дата':'Date','Регион':'Region','Заражений':'Confirmed','Выздоровлений':'Recovered',
                               'Смертей':'Deaths','Смертей за день':'Day_deaths',
                               'Заражений за день':'Day_confirmed','Выздоровлений за день':'Day_recovered'},inplace = True)


for i in range(yandex_russia.shape[0]):
    a= yandex_russia.Date[i].split('.')
    yandex_russia.Date[i]=datetime.date(int(a[2]),int(a[1]),int(a[0]))
    
   
yandex_russia.tail(10)

,Date,Region,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered
41538,2020-05-28,Амурская обл.,433,139,5,0,19,6
41539,2021-06-20,Калмыкия,21202,20507,398,2,27,21
41540,2020-04-22,Северная Осетия,211,8,2,0,34,0
41541,2020-07-31,Чукотский АО,152,148,1,0,0,0
41542,2021-03-12,Калужская обл.,30962,28716,272,1,93,90
41543,2020-04-24,Камчатский край,84,5,0,0,6,0
41544,2020-10-13,Брянская обл.,10446,8264,41,0,86,24
41545,2021-05-06,Амурская обл.,22377,21565,285,0,13,17
41546,2021-02-09,Алтайский край,40611,36341,1296,13,174,170
41547,2021-04-20,Забайкальский край,41970,40480,615,2,36,32


In [3]:
df3_temp = yandex_russia.groupby('Date').sum()
df3_temp['Region']='Россия'

y5= pd.concat([yandex_russia,df3_temp.reset_index()],axis = 0)
y5.reset_index()
yandex_russia = y5
yandex_russia.tail(10)

,Date,Region,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered
479,2021-07-04,Россия,5610941,5068901,137925,663,25142,15484
480,2021-07-05,Россия,5635294,5083441,138579,654,24353,14540
481,2021-07-06,Россия,5658672,5101852,139316,737,23378,18411
482,2021-07-07,Россия,5682634,5121919,140041,725,23962,20067
483,2021-07-08,Россия,5707452,5143255,140775,734,24818,21336
484,2021-07-09,Россия,5733218,5165087,141501,726,25766,21832
485,2021-07-10,Россия,5758300,5182837,142253,752,25082,17750
486,2021-07-11,Россия,5783333,5200219,143002,749,25033,17382
487,2021-07-12,Россия,5808473,5216648,143712,710,25140,16429
488,2021-07-13,Россия,5833175,5236214,144492,780,24702,19566


In [4]:
# dt = datetime.datetime.today()
# today = datetime.date(dt.year,dt.month,dt.day)
# ystday = datetime.date(today.year,today.month,today.day-1)
# yst2day = datetime.date(today.year,today.month,today.day-2)
# yst3day = datetime.date(today.year,today.month,today.day-3)

# print(today)
# print(ystday)
# print(yst2day)
# print(yst3day)
myday = list(yandex_russia.tail(1).Date)[0]

print(yandex_russia.iloc[-1,0])
print('Выбран день: ',myday)

2021-07-13
Выбран день:  2021-07-13


In [5]:
X = pd.DataFrame()
regions_dict = {}
yandex_russia['Rt'] = 1
yandex_russia['Rt2'] = 1
yandex_russia['Rate_conf'] = 0
yandex_russia['Rate_ill'] = 0

temp_df = yandex_russia.groupby(['Region','Date']).sum()

# Ниже enumerate  для составления словаря из имен регионов {i:j} , для расчетов можно было использовать только j

for i,j in enumerate(temp_df.index.get_level_values(0).unique()):
    temp_df2 =  temp_df.loc[j]
    temp1 = temp_df2['Day_confirmed'].rolling(4).sum()/(temp_df2["Day_confirmed"].rolling(8).sum()-

                                                              temp_df2["Day_confirmed"].rolling(4).sum())
    temp2 = temp_df2['Confirmed'].rolling(4).sum()/(temp_df2['Confirmed'].rolling(8).sum()-

                                                              temp_df2['Confirmed'].rolling(4).sum())
    temp_df2['Rt'] = round(temp1,3)
    temp_df2['Rt2'] = round(temp2,3)
    temp_df2['Region'] = j
    temp_df2['Remaining_ill']=temp_df2['Confirmed']-temp_df2['Recovered']-temp_df2['Deaths']
    temp_df2['MA7_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=7).mean()
    temp_df2['MA14_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=14).mean()
    temp_df2['MA7_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=7).mean()
    temp_df2['MA14_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=14).mean()
    
    temp_df2['Change_dayconf'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_dayconf'].iloc[k]=temp_df2['Day_confirmed'].iloc[k]-temp_df2['Day_confirmed'].iloc[k-1]
    temp_df2['Change_remill'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_remill'].iloc[k]=temp_df2['Remaining_ill'].iloc[k]-temp_df2['Remaining_ill'].iloc[k-1]
    
    

   #Генератор DF  на каждый регион(сидит в цикле):
   # globals()['x' + str(i)] = temp_df2.reset_index()
    
    X = pd.concat([X,temp_df2.reset_index()],axis = 0)
    regions_dict[i]=j
    
print(regions_dict.items())
    
X = X.reset_index(drop=True)
X.tail()


dict_items([(0, 'Адыгея'), (1, 'Алтай'), (2, 'Алтайский край'), (3, 'Амурская обл.'), (4, 'Архангельская обл.'), (5, 'Астраханская обл.'), (6, 'Башкортостан'), (7, 'Белгородская обл.'), (8, 'Брянская обл.'), (9, 'Бурятия'), (10, 'Владимирская обл.'), (11, 'Волгоградская обл.'), (12, 'Вологодская обл.'), (13, 'Воронежская обл.'), (14, 'Дагестан'), (15, 'Еврейская АО'), (16, 'Забайкальский край'), (17, 'Ивановская обл.'), (18, 'Ингушетия'), (19, 'Иркутская обл.'), (20, 'Кабардино-Балкария'), (21, 'Калининградская обл.'), (22, 'Калмыкия'), (23, 'Калужская обл.'), (24, 'Камчатский край'), (25, 'Карачаево-Черкессия'), (26, 'Карелия'), (27, 'Кемеровская обл.'), (28, 'Кировская обл.'), (29, 'Коми'), (30, 'Костромская обл.'), (31, 'Краснодарский край'), (32, 'Красноярский край'), (33, 'Крым'), (34, 'Курганская обл.'), (35, 'Курская обл.'), (36, 'Ленинградская обл.'), (37, 'Липецкая обл.'), (38, 'Магаданская обл.'), (39, 'Марий Эл'), (40, 'Мордовия'), (41, 'Москва'), (42, 'Московская обл.'), (4

,Date,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered,Rt,Rt2,Rate_conf,Rate_ill,Region,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
42032,2021-07-09,45318,41563,747,5,202,77,1.193,1.015,0,0,Ярославская обл.,3008,170.428571,149.071429,2687.285714,2395.357143,12,120
42033,2021-07-10,45525,41643,752,5,207,80,1.244,1.016,0,0,Ярославская обл.,3130,177.571429,155.285714,2791.714286,2481.714286,5,122
42034,2021-07-11,45729,41730,757,5,204,87,1.247,1.017,0,0,Ярославская обл.,3242,184.714286,161.500000,2899.142857,2572.285714,-3,112
42035,2021-07-12,45929,41813,763,6,200,83,1.196,1.017,0,0,Ярославская обл.,3353,191.571429,167.500000,3009.428571,2666.571429,-4,111
42036,2021-07-13,46124,41900,768,5,195,87,1.104,1.018,0,0,Ярославская обл.,3456,196.428571,172.785714,3121.571429,2764.000000,-5,103


In [6]:
mask = X.Region == 'Россия'
temp_df = X[mask][['Date','Confirmed','Remaining_ill']]
temp_df.index = temp_df.Date
temp_df.drop(columns =['Date'], inplace = True)
print(temp_df.tail(2))
# print(temp_df.index.get_level_values(0))
# temp_df.loc[datetime.date(2020,6,15),'Россия'].Confirmed
# for i in temp_df.index.get_level_values(0).unique()[-2:]:
#     print(temp_df.loc[i,'Россия'].Confirmed)
rus_date = list(temp_df.index)
rus_conf = list(temp_df.Confirmed)
rus_ill = list(temp_df.Remaining_ill)
rus_dict_conf = dict(zip(rus_date,rus_conf))
rus_dict_ill = dict(zip(rus_date,rus_ill))

print(len(X))
print(X.shape)
print(X.loc[0,'Date'])
for i in range(len(X)):
    X.loc[i,'Rate_conf']=100*X.loc[i,'Confirmed']/rus_dict_conf.get(X.loc[i,'Date'])
    X.loc[i,'Rate_ill']=100*X.loc[i,'Remaining_ill']/rus_dict_ill.get(X.loc[i,'Date'])


            Confirmed  Remaining_ill
Date                                
2021-07-12    5808473         448113
2021-07-13    5833175         452469
42037
(42037, 19)
2020-03-12


In [7]:
mask = (X.Date>=datetime.date(2020,6,13))&(X.Region.isin(['Москва','Россия']))

temp=X[mask]
temp[['Date','Confirmed','Region','Rate_conf','Rate_ill']].tail(20)

,Date,Confirmed,Region,Rate_conf,Rate_ill
27352,2021-06-24,5388695,Россия,100.0,100.0
27353,2021-06-25,5409088,Россия,100.0,100.0
27354,2021-06-26,5430753,Россия,100.0,100.0
27355,2021-06-27,5451291,Россия,100.0,100.0
27356,2021-06-28,5472941,Россия,100.0,100.0
27357,2021-06-29,5493557,Россия,100.0,100.0
27358,2021-06-30,5514599,Россия,100.0,100.0
27359,2021-07-01,5538142,Россия,100.0,100.0
27360,2021-07-02,5561360,Россия,100.0,100.0
27361,2021-07-03,5585799,Россия,100.0,100.0


In [8]:
# russia_regions = pd.read_csv('regions-info.csv')

# russia_regions[(russia_regions.Region =='Московская область')|(russia_regions.Region =='Москва')|
#               (russia_regions.Region =='Санкт-Петербург')].tail()

In [9]:
X.to_csv('X.csv',index=False)
